In [1]:
!pip install onnx-graphsurgeon

  Using cached protobuf-6.31.1-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 4.3 MB/s eta 0:00:0000:0100:01
Using cached protobuf-6.31.1-cp39-abi3-manylinux2014_x86_64.whl (321 kB)


In [2]:
import onnx
import onnx_graphsurgeon as gs
import numpy as np
import os

In [3]:
input_onnx_path = "../../../tests/data/onnx/decoder_model_merged.onnx"
output_onnx_path = "../../../tests/data/extracted_layer.onnx"  # Output path for the extracted layer

# Load the ONNX model
graph = gs.import_onnx(onnx.load(input_onnx_path))

# List all node names to find the one you want to extract
print("All node names in the model:")
for node in graph.nodes:
    print(node.name, node.op)

# --- USER: Set this to the name of the layer you want to extract ---
target_node_name = "/model/layers.0/attn/q_proj/MatMul"  # Example, change as needed

# Find the node
target_node = None
for node in graph.nodes:
    if node.name == target_node_name:
        target_node = node
        break

if target_node is None:
    raise ValueError(f"Node {target_node_name} not found in the model.")

print(">>>", target_node.outputs)
print("<<<", target_node.inputs)

# Identify the activation input (not weights/biases)
activation_input = None
for inp in target_node.inputs:
    if inp.name in [i.name for i in graph.inputs]:
        activation_input = inp
        break
if activation_input is None:
    # If not found in graph.inputs, assume the first input is the activation
    activation_input = target_node.inputs[0]

# Set the graph input to the activation input only
graph.inputs = [activation_input]
# Set the graph output to the output of the target node
graph.outputs = target_node.outputs

# Clean up the graph (removes unused nodes and initializers)
graph.cleanup()

# Export the new ONNX model
onnx.save(gs.export_onnx(graph), output_onnx_path)

print(f"Extracted layer saved to {output_onnx_path}")

All node names in the model:
/model/attn_mask_reformat/attn_mask_subgraph/Shape Shape
/model/layers.0/input_layernorm/LayerNorm SimplifiedLayerNormalization
/model/attn_mask_reformat/attn_mask_subgraph/ReduceSum ReduceSum
/model/attn_mask_reformat/attn_mask_subgraph/Gather Gather
/model/layers.0/attn/q_proj/MatMul MatMul
/model/layers.0/attn/k_proj/MatMul MatMul
/model/layers.0/attn/v_proj/MatMul MatMul
/model/attn_mask_reformat/attn_mask_subgraph/Sub Sub
/model/attn_mask_reformat/attn_mask_subgraph/Gather/Cast Cast
/model/layers.0/attn/q_rotary/RotaryEmbedding RotaryEmbedding
/model/layers.0/attn/k_rotary/RotaryEmbedding RotaryEmbedding
/model/attn_mask_reformat/attn_mask_subgraph/Sub/Cast Cast
/model/layers.0/attn/GroupQueryAttention GroupQueryAttention
/model/layers.0/attn/o_proj/MatMul MatMul
/model/layers.0/post_attention_layernorm/SkipLayerNorm SkipSimplifiedLayerNormalization
/model/layers.0/mlp/gate_proj/MatMul MatMul
/model/layers.0/mlp/up_proj/MatMul MatMul
/model/layers.0/ml